In [1]:
import pystan
from pystan import StanModel

%matplotlib inline
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.cross_validation import train_test_split

from sklearn.datasets import fetch_mldata
import scipy.io as sio

/home/uras/miniconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# mnist = fetch_mldata('MNIST original', data_home='.')
### to download MNIST data, uncomment the line above and run ###
mnist = sio.loadmat('mldata/mnist-original.mat')
data = mnist['data'].T
label = mnist['label'].T

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=1.0/7)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [7]:
model = """
data {
    real init_1[784, 100];
    real init_2[100, 10];
    real init_out[10];
    real input[60000, 784];
    real output[60000, 1];
}
parameters {
    real weights_in_1[784, 100];
    real weights_1_2[100, 10];
    real weights_2_out[10];
    real out[10];
}
transformed parameters{
    matrix[784, 100] act_1 = tanh(dot_product(input, weights_in_1))
    matrix[100, 10] act_2 = tanh(dot_product(act_1, weights_1_2))
    vector[10] act_out = ilogit(dot_product(act_2, weights_2_out))
}
model {
    for (i in 1:784)
        for (j in 1:100)
            weights_in_1[i,j] ~ normal(0, 1);
            
    for (i in 1:784)
        for (j in 1:100)
            weights_1_2[i,j] ~ normal(0, 1);
    
    for (i in 1:10)
            weights_2_out[i] ~ normal(0, 1);
        
    for (i in 1:10)
        out[i] ~ dcat(act_out[])
}
"""

In [8]:
n_hidden = 100
n_classes = 10

# Initialize random weights between each layer
init_1 = np.random.randn(data.shape[1], n_hidden).astype('float32')
init_2 = np.random.randn(n_hidden, n_classes).astype('float32')
init_out = np.random.randn(n_classes).astype('float32')

data_dict = {'init_1': init_1, 'init_2': init_2 ,'init_out' : init_out, 'input' : X_train, 'output' : Y_train}

fit_model = StanModel(model_code=model)

fit = fit_model.sampling(data=data_dict, iter=1000, chains=1)
#fit_eps = pystan.stan(model_code=model, data=data_dict, iter=1000, chains=1)
print(fit)

ValueError: Failed to parse Stan model 'anon_model_54381e03b96ae1a1f2eec9234a3aab72'. Error message:
SYNTAX ERROR, MESSAGE(S) FROM PARSER:

No matches for: 

  dot_product(real[,], real[,])

Available argument signatures for dot_product:

  dot_product(vector, vector)
  dot_product(row vector, row vector)
  dot_product(vector, row vector)
  dot_product(row vector, vector)
  dot_product(real[], real[])

  error in 'unkown file name' at line 16, column 67
  -------------------------------------------------
    14: }
    15: transformed parameters{
    16:     matrix[784, 100] act_1 = tanh(dot_product(input, weights_in_1))
                                                                          ^
    17:     matrix[100, 10] act_2 = tanh(dot_product(act_1, weights_1_2))
  -------------------------------------------------

